In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_table("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/omics/olink_data.txt")
df = df[df["ins_index"]==0]
codings = pd.read_csv('/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/codings/coding143.tsv', sep='\t')
codings[['protein_abbr','protein_name']] = codings['meaning'].str.split(';',expand=True).rename(columns={0:'protein_abbr',1:'protein_name'})
codings.rename(columns={"coding":"protein_id"},inplace=True)

In [3]:
pd_patients = pd.read_csv('PD_filtered.csv').iloc[:,1:]

In [4]:
withdrawn = pd.read_csv("../Feature_selection_ligh/withdrawnparticipants.csv")
first_col_name = withdrawn.columns[0]
ids_withdrawn_list = [first_col_name] + withdrawn.iloc[:, 0].tolist()

In [5]:
df_merged = pd.merge(codings, df, on=["protein_id"])
df_merged.rename(columns={"result":"Expression levels"},inplace=True)
df_merged = df_merged[df_merged["ins_index"] == 0]

In [6]:
df_merged = df_merged[["eid","protein_abbr","Expression levels"]]

In [7]:
df_merged = df_merged[~df_merged["eid"].isin(ids_withdrawn_list)]

In [8]:
df_pivot = df_merged.pivot_table(index=['eid'], columns='protein_abbr', values='Expression levels').reset_index()

In [9]:
df_pd = df_pivot[df_pivot["eid"].isin(set(pd_patients["eid"]))]

In [10]:
df_pd

protein_abbr,eid,A1BG,AAMDC,AARSD1,ABCA2,ABHD14B,ABL1,ABO,ABRAXAS2,ACAA1,...,ZFYVE19,ZHX2,ZNF174,ZNF75D,ZNF830,ZNRD2,ZNRF4,ZP3,ZP4,ZPR1
23,1001877,NaN,NaN,0.75865,NaN,0.24115,1.18815,NaN,NaN,0.86420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,1006331,NaN,NaN,-0.41365,NaN,0.14475,0.63725,NaN,NaN,0.85050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,1006433,0.0252,-0.76870,-0.99790,0.1610,-0.31130,-0.49470,0.1322,0.1899,-0.48680,...,0.3285,-0.0932,-0.2388,0.3158,-0.25310,-0.18510,0.5979,0.2022,0.1142,0.5349
193,1017084,-0.1199,-0.28155,-0.45555,-0.3075,-0.14835,0.44150,-1.5782,0.0692,-0.60595,...,-0.5916,0.0685,-0.0980,0.4265,-0.09180,0.95985,-0.5603,-5.3629,-0.4625,0.6897
194,1017311,0.1832,0.70115,0.51250,-0.1828,0.34410,1.18210,0.0094,0.3653,-0.06110,...,0.7774,0.0791,NaN,0.0477,0.43655,1.24680,0.9547,0.9646,-0.5634,-0.0970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52520,5980240,-0.2897,-0.09615,-0.08605,-0.1062,0.02365,-0.47740,1.6817,-0.2978,-0.65170,...,0.1063,-0.0141,0.0097,0.0795,-0.45570,-0.34595,1.5277,-4.9319,-0.0583,0.3069
52585,5985610,0.0990,0.23815,NaN,0.5539,0.28650,NaN,0.7130,0.3429,1.23820,...,0.9843,-0.1006,0.0217,0.2544,0.74250,0.96645,0.1570,-5.4014,0.1328,3.1415
52772,6004650,NaN,NaN,1.05115,NaN,0.13055,-0.84475,NaN,NaN,-0.05870,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52774,6004981,-0.0064,-0.81325,-1.15345,-0.2555,-1.05625,-0.69220,-2.1061,-0.7146,-0.40315,...,-1.8254,0.4152,0.1470,0.5790,0.12650,-0.33825,-0.2576,0.3376,-0.0770,0.3580


In [11]:
missing_fraction = df_pd.isnull().mean()
cols_missing_over_30 = missing_fraction[missing_fraction > 0.3].index.tolist()



In [12]:
df_pd_filtered = df_pd.set_index("eid").drop(cols_missing_over_30, axis=1)


In [13]:
columns_tokeep = df_pd_filtered.columns.unique().tolist()

In [14]:
with open("columns_30pd.txt", "w") as f:
    for item in columns_tokeep:
        f.write(f"{item}\n")

In [15]:
len(columns_tokeep)

1461